In [60]:
# historical avg
import vaex
from datetime import datetime


# https://data-newbie.tistory.com/583
def multipe_column_join(x_left, x_right, keys_left, keys_right, how="left"):
    # can add more checks like, vx_df[idx] cannot be an object,
    # allow to pass only one set of keys

    if len(keys_left) == 1:
        join_result = x_left.join(
            x_right, left_on=keys_left[0], right_on=keys_right[0], how=how, allow_duplication=True
        )

    elif len(keys_left) > 1:
        for idx, right_key in enumerate(keys_right):
            add_group = x_right[right_key]

            if not add_group.dtype == str:
                add_group = add_group.astype(str)

            if idx == 0:
                x_right["group_right"] = add_group
            else:
                x_right["group_right"] = x_right["group_right"] + "_" + add_group
        for idx, left_key in enumerate(keys_left):
            add_group = x_left[left_key]
            if not add_group.dtype == str:
                add_group = add_group.astype(str)

            if idx == 0:
                x_left["group_left"] = add_group
            else:
                x_left["group_left"] = x_left["group_left"] + "_" + add_group

        x_right.drop(keys_right, inplace=True)
        join_result = x_left.join(x_right, left_on="group_left", right_on="group_right", how=how)
        join_result.drop(["group_left","group_right"],inplace=True)
    return join_result


vx_df = vaex.open('./output/one_route.hdf5')

# temp 
def get_dt(dt, format):
    return datetime.strptime(str(dt), format)
def apply_dow(start_date, start_time, expected_time):
    date = get_dt(start_date, "%Y%m%d")
    if get_dt(start_time, "%H:%M:%S") > get_dt(expected_time, "%H:%M:%S"):
        return (date + timedelta(days=1)).weekday()
    return date.weekday()
vx_df['arr_dow'] = vx_df.apply(apply_dow, ["start_date", "start_time", "arrival_time"])

cols = ['trip_id', 'stop_id', 'arr_dow']
trip_stops = vx_df.groupby(cols, agg=[vaex.agg.mean('arrival')])

joined_df = multipe_column_join(vx_df, trip_stops, cols, cols)
joined_df

#,trip_id,start_date,start_time,stop_sequence,departure,arrival,timestamp,stop_id,arrival_time,departure_time,shape_dist_traveled,direction,route_id,lat,lon,direction_angle,shape_dist_between,p_avg_vol,arr_dow,arrival_mean
0,8213.2.60-27-b12-1.151.O,20210130,15:40:00,78,360,360,2021-01-30 16:49:33,8230DB005008,16:54:47,16:54:47,24147.77,0,60-27-b12-1,53.2904501890302,-6.3693569938866,-118.2101957090697,312.6899999999987,204.08747428810207,5,270.0
1,8112.2.60-27-b12-1.149.I,20210130,16:00:00,47,120,120,2021-01-30 16:39:06,8220DB001358,16:45:44,16:45:44,16067.82,1,60-27-b12-1,53.3443047133873,-6.26288938939078,61.73872317468925,441.1900000000005,19.81116220357116,5,60.0
2,17753.4.60-27-b12-1.149.I,20210129,12:30:00,1,0,0,2021-01-29 12:02:01,8230DB002353,12:30:00,12:30:00,0.0,1,60-27-b12-1,53.279150529518,-6.40175897234439,61.73872317468925,0.0,202.96186700062577,4,0.0
3,8112.2.60-27-b12-1.149.I,20210130,16:00:00,83,180,180,2021-01-30 17:15:20,8220DB001267,17:20:11,17:20:11,26472.7,1,60-27-b12-1,53.4008555065983,-6.20024887014065,61.73872317468925,323.9900000000016,63.9830036801728,5,90.0
4,17586.4.60-27-b12-1.149.I,20210126,11:00:00,72,0,0,2021-01-26 11:34:11,8220DB004695,12:06:42,12:06:42,23758.73,1,60-27-b12-1,53.3929830768232,-6.1973482266491,61.73872317468925,336.2200000000012,36.54685743087244,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"43,739",17672.4.60-27-b12-1.149.I,20210122,17:20:00,60,0,0,2021-01-22 18:22:24,8220DB000669,18:23:16,18:23:16,19788.3,1,60-27-b12-1,53.3663170047536,-6.2286789053178,61.73872317468925,360.6100000000006,91.48522532185576,4,0.0
"43,740",17616.4.60-27-b12-1.149.I,20210127,12:10:00,84,120,120,2021-01-27 13:24:13,8220DB001268,13:29:43,13:29:43,26661.19,1,60-27-b12-1,53.399894267545,-6.19789824293475,61.73872317468925,188.489999999998,79.35148749915943,2,120.0
"43,741",8072.2.60-27-b12-1.149.I,20210123,11:10:00,26,300,300,2021-01-23 11:26:13,8230DB002377,11:33:07,11:33:07,9710.88,1,60-27-b12-1,53.3160470777079,-6.33416048619781,61.73872317468925,947.83,105.32452282475197,5,300.0
"43,742",17593.4.60-27-b12-1.149.I,20210125,15:10:00,9,-60,-60,2021-01-25 15:24:31,8230DB007062,15:16:44,15:16:44,2590.62,1,60-27-b12-1,53.2848097660929,-6.39349379003628,61.73872317468925,287.7999999999997,210.66928960605887,0,-60.0


In [53]:
joined_df.extract().column_names

['trip_id',
 'start_date',
 'start_time',
 'stop_sequence',
 'departure',
 'arrival',
 'timestamp',
 'stop_id',
 'arrival_time',
 'departure_time',
 'shape_dist_traveled',
 'direction',
 'route_id',
 'lat',
 'lon',
 'direction_angle',
 'shape_dist_between',
 'p_avg_vol',
 'arr_dow',
 '__group_left_1',
 '__group_left',
 '__trip_id',
 '__stop_id',
 '__arr_dow',
 'arrival_mean',
 '__arrival_mean_1',
 '__group_right_1',
 '__group_right']

In [27]:
trip_stops.trip_id.dtype == str

True

In [78]:
# join multiple vaex columns
def multipe_column_join(x_left, x_right, keys_left:list, keys_right:list, how="left"):

    assert type(keys_left) == list and type(keys_right) == list, f'keys must be list and list, not {type(keys_left)} and {type(keys_right)}'
    assert len(keys_left) == len(keys_right), f'lenghts of left and right keys dont match, left: {len(keys_left)} right: {len(keys_right)}'

    if len(keys_left) == 1:
        join_result = x_left.join(
            x_right, left_on=keys_left[0], right_on=keys_right[0], how=how, allow_duplication=True
        )

    elif len(keys_left) > 1:
        for idx, zp in enumerate(zip(keys_left, keys_right)):
            left_key, right_key = zp
            
            add_left = x_left[left_key]
            add_right = x_right[right_key]

            if not add_left.dtype == str:
                add_left = add_left.astype(str)
            if not add_right.dtype == str:
                add_right = add_right.astype(str)

            if idx == 0:
                x_right["group_right"] = add_right
                x_left["group_left"] = add_left
            else:
                x_right["group_right"] = x_right["group_right"] + "_" + add_right
                x_left["group_left"] = x_left["group_left"] + "_" + add_left

        x_right.drop(keys_right, inplace=True)
        join_result = x_left.join(x_right, left_on="group_left", right_on="group_right", how=how)
        join_result.drop(["group_left", "group_right"], inplace=True)
    return join_result

# temp 
vx_df = vaex.open('./output/one_route.hdf5')
def get_dt(dt, format):
    return datetime.strptime(str(dt), format)
def apply_dow(start_date, start_time, expected_time):
    date = get_dt(start_date, "%Y%m%d")
    if get_dt(start_time, "%H:%M:%S") > get_dt(expected_time, "%H:%M:%S"):
        return (date + timedelta(days=1)).weekday()
    return date.weekday()
vx_df['arr_dow'] = vx_df.apply(apply_dow, ["start_date", "start_time", "arrival_time"])

cols = ['trip_id', 'stop_id', 'arr_dow']
trip_stops = vx_df.groupby(cols, agg=[vaex.agg.mean('arrival')])

multipe_column_join(vx_df, trip_stops, cols, cols)

#,trip_id,start_date,start_time,stop_sequence,departure,arrival,timestamp,stop_id,arrival_time,departure_time,shape_dist_traveled,direction,route_id,lat,lon,direction_angle,shape_dist_between,p_avg_vol,arr_dow,arrival_mean
0,8213.2.60-27-b12-1.151.O,20210130,15:40:00,78,360,360,2021-01-30 16:49:33,8230DB005008,16:54:47,16:54:47,24147.77,0,60-27-b12-1,53.2904501890302,-6.3693569938866,-118.2101957090697,312.6899999999987,204.08747428810207,5,270.0
1,8112.2.60-27-b12-1.149.I,20210130,16:00:00,47,120,120,2021-01-30 16:39:06,8220DB001358,16:45:44,16:45:44,16067.82,1,60-27-b12-1,53.3443047133873,-6.26288938939078,61.73872317468925,441.1900000000005,19.81116220357116,5,60.0
2,17753.4.60-27-b12-1.149.I,20210129,12:30:00,1,0,0,2021-01-29 12:02:01,8230DB002353,12:30:00,12:30:00,0.0,1,60-27-b12-1,53.279150529518,-6.40175897234439,61.73872317468925,0.0,202.96186700062577,4,0.0
3,8112.2.60-27-b12-1.149.I,20210130,16:00:00,83,180,180,2021-01-30 17:15:20,8220DB001267,17:20:11,17:20:11,26472.7,1,60-27-b12-1,53.4008555065983,-6.20024887014065,61.73872317468925,323.9900000000016,63.9830036801728,5,90.0
4,17586.4.60-27-b12-1.149.I,20210126,11:00:00,72,0,0,2021-01-26 11:34:11,8220DB004695,12:06:42,12:06:42,23758.73,1,60-27-b12-1,53.3929830768232,-6.1973482266491,61.73872317468925,336.2200000000012,36.54685743087244,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"43,739",17672.4.60-27-b12-1.149.I,20210122,17:20:00,60,0,0,2021-01-22 18:22:24,8220DB000669,18:23:16,18:23:16,19788.3,1,60-27-b12-1,53.3663170047536,-6.2286789053178,61.73872317468925,360.6100000000006,91.48522532185576,4,0.0
"43,740",17616.4.60-27-b12-1.149.I,20210127,12:10:00,84,120,120,2021-01-27 13:24:13,8220DB001268,13:29:43,13:29:43,26661.19,1,60-27-b12-1,53.399894267545,-6.19789824293475,61.73872317468925,188.489999999998,79.35148749915943,2,120.0
"43,741",8072.2.60-27-b12-1.149.I,20210123,11:10:00,26,300,300,2021-01-23 11:26:13,8230DB002377,11:33:07,11:33:07,9710.88,1,60-27-b12-1,53.3160470777079,-6.33416048619781,61.73872317468925,947.83,105.32452282475197,5,300.0
"43,742",17593.4.60-27-b12-1.149.I,20210125,15:10:00,9,-60,-60,2021-01-25 15:24:31,8230DB007062,15:16:44,15:16:44,2590.62,1,60-27-b12-1,53.2848097660929,-6.39349379003628,61.73872317468925,287.7999999999997,210.66928960605887,0,-60.0


In [74]:
vx_df.dtypes

trip_id                <class 'str'>
start_date                     int64
start_time             <class 'str'>
stop_sequence                  int64
departure                      int64
arrival                        int64
timestamp              <class 'str'>
stop_id                <class 'str'>
arrival_time           <class 'str'>
departure_time         <class 'str'>
shape_dist_traveled          float64
direction                      int64
route_id               <class 'str'>
lat                          float64
lon                          float64
direction_angle              float64
shape_dist_between           float64
p_avg_vol                    float64
arr_dow                        int64
group_left             <class 'str'>
dtype: object

In [12]:
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import Parse

feed = gtfs_realtime_pb2.FeedMessage()
entity_data = []


Parse(open('./test.json', 'r').read(), feed)

count = 0
for entity in feed.entity:
    if entity.HasField("trip_update"):
        stop_time_update = entity.trip_update.stop_time_update
        count += len(stop_time_update)
    count += 1
                
count

9761